In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/tweet-sentiment-extraction/train.csv
/kaggle/input/tweet-sentiment-extraction/test.csv
/kaggle/input/tweet-sentiment-extraction/sample_submission.csv


In [2]:
df = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/train.csv").astype(str)
test = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/test.csv").astype(str)
submission = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/sample_submission.csv").astype(str)


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import re
import os
import spacy

#https://pypi.org/project/rake-nltk/
!pip install rake_nltk

#import rake_nltk
from rake_nltk import Metric, Rake
r = Rake()

#https://github.com/vi3k6i5/flashtext
from flashtext import KeywordProcessor

#TextRank
#https://towardsdatascience.com/textrank-for-keyword-extraction-by-python-c0bae21bcec0
!pip install pytextrank
import pytextrank

#Spacy
import spacy
nlp = spacy.load('en_core_web_sm')


#Bar
from tqdm import tqdm, tqdm_pandas
tqdm(tqdm())
import matplotlib.pyplot as plt


  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7819 sha256=675f64a7d2e3e0851fab9c29c22c95b11716f016f0a652001dc56ba552fe0499
  Stored in directory: /root/.cache/pip/wheels/ad/4f/6b/a5e7778f5c084d8b7c2c5fe9d37eb806f5ff5fee5e8ee17342
Successfully built rake-nltk


0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


## 1. Pre-processing 

In [4]:
### Own Stop words
own_stop_word = ['i','we','are','and']
### Spacy Lemma 
def spacy_lemma_text(text):
    doc = nlp(text)
    tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
    tokens = [tok for tok in tokens if tok not in own_stop_word ]
    tokens = ' '.join(tokens)
    return tokens

### Remove URL
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

In [5]:
df['text_clean'] = df['text'].apply(remove_URL)
df['text_clean'] = df['text_clean'].apply(spacy_lemma_text)
print("Train Cleaning - Done")
test['text_clean'] = test['text'].apply(remove_URL)
test['text_clean'] = test['text_clean'].apply(spacy_lemma_text)
print("Test Cleaning - Done")


Train Cleaning - Done
Test Cleaning - Done


In [6]:
test['sentiment'].value_counts()

neutral     1430
positive    1104
negative    1001
Name: sentiment, dtype: int64

## Sentence Lenght Analysis


In [7]:
from IPython.core.display import display, HTML
import plotly.graph_objects as go
test ['length'] = test['text'].apply(len)

data = [
    go.Box(
        y=test[test['sentiment']=='positive']['length'],
        name='positive'
    ),
    go.Box(
        y=test[test['sentiment']=='negative']['length'],
        name='negative'
    ),
    go.Box(
        y=test[test['sentiment']=='neutral']['length'],
        name='neutral'
    ),

]
layout = go.Layout(
    title = 'Sentiment Vs Comment Lenght (Before pre-processing)'
)
fig1 = go.Figure(data=data, layout=layout)


test ['length'] = test['text_clean'].apply(len)

data = [
    go.Box(
        y=test[test['sentiment']=='positive']['length'],
        name='positive'
    ),
    go.Box(
        y=test[test['sentiment']=='negative']['length'],
        name='negative'
    ),
    go.Box(
        y=test[test['sentiment']=='neutral']['length'],
        name='neutral'
    ),

]
layout = go.Layout(
    title = 'Sentiment Vs Comment Lenght (After pre-processing)'
)
fig2 = go.Figure(data=data, layout=layout)


In [8]:
fig1.show()

In [9]:
fig2.show()

## 2 . Spacy Aspect-Based Opinion Mining is to extract product's aspects and the associated user opinions from the user text review

In [10]:
aspect_terms = []
for review in nlp.pipe(test.text_clean):
    chunks = [(chunk.root.text) for chunk in review.noun_chunks if chunk.root.pos_ == 'NOUN']
    aspect_terms.append(' '.join(chunks))
    
test['Aspect_Terms'] = aspect_terms    

In [11]:
test.sample(5)

,textID,text,sentiment,text_clean,length,Aspect_Terms
1876,ebae3ed394,"OK, time to head into Urban Outfitters to see ...",neutral,"ok , time to head into urban outfitters to see...",131,outfitters rain day
3128,6cd9991cca,"oic, where in Cavite? no sorry im not from Am...",neutral,"oic , where in cavite ? no sorry be not from ...",114,profile word
532,62ed3d6141,What's wrong with Dr.Phil? I shamefully loved...,neutral,what be wrong with dr . phil ? shamefully lov...,76,show
3051,d3a576d221,"you asked about my SF schedule, dahling...may...",neutral,"ask about sf schedule , dahle ... maybe next ...",50,time
3168,af087c9deb,youre diabetic? so is my dad. &amp; i mig...,negative,be diabetic ? so be dad . & amp ; may be to...,110,dad sugar day


## 3. Sentiment Word Extract

In [12]:
sentiment_terms = []

for review in nlp.pipe(test['text_clean']):
        if review.is_parsed:
            sentiment_terms.append(' '.join([token.lemma_ for token in review if (not token.is_stop and not token.is_punct and (token.pos_ == "ADJ" or token.pos_ == "VERB"))]))
        else:
            sentiment_terms.append('')  
            
test['Sentiment_terms'] = sentiment_terms  

In [13]:
test.sample(5)

,textID,text,sentiment,text_clean,length,Aspect_Terms,Sentiment_terms
2538,4fed31099f,@_Anix_ I bought cheesecake ice cream last wee...,negative,"@_anix _ buy cheesecake ice cream last week , ...",87,cream,buy think good suck
909,5f72b7df41,Now have windows 7 up and running....very very...,positive,now have window 7 up run .... very very happy ...,76,window run sum feature,happy wicked new
2339,10022832ae,I'm suffering from restless legs and butt synd...,negative,be suffer from restless leg butt syndrome .,43,syndrome,suffer
1846,31d260e718,There's no pause button in the game of life.,neutral,there be no pause button in the game of life .,47,button game life,
3214,12cfea907c,Already misses,negative,already miss,12,,


## 4. PyTextRank is a Python implementation of TextRank as a spaCy extension


In [14]:
import spacy
import pytextrank
nlp = spacy.load('en_core_web_sm')
tr = pytextrank.TextRank()
nlp.add_pipe(tr.PipelineComponent, name='textrank', last=True)

In [15]:
pytext_key = []

for text in test['text_clean']:
    text = nlp(text)
    t = text._.phrases
    pytext_key.append(t)
    
test['Pytextrank_keyword'] = pytext_key  

In [16]:
test['Pytextrank_keyword'] = test['Pytextrank_keyword'].agg(lambda x: ','.join(map(str, x)))


In [17]:
test.sample(5)

,textID,text,sentiment,text_clean,length,Aspect_Terms,Sentiment_terms,Pytextrank_keyword
3083,161bfc7c8a,"yep - three things: a good haircut, the abili...",positive,"yep - three thing : a good haircut , the abil...",95,thing haircut ability value,good listen,"a good haircut,the value,the ability, yep - th..."
315,5b70f8bc73,"maybe i should take up drinking again, feel as...",negative,"maybe should take up drink again , feel as if ...",96,,drink feel massive,today
2661,e7f2b44a1f,I totally forgot my phone at home this morning,negative,totally forget phone at home this morning,41,phone home,forget,"home,phone,this morning"
1183,e6f7199479,Rofl u did? Hahaha! Lemme tell Salman Munir ...,positive,rofl u do ? hahaha ! lemme tell salman munir...,54,p,tell,"salman munir,u,lemme,salman,= p = p, rofl,hahaha"
2306,2b6ec32c28,_WOCKEEZ I'm missin u guys by a day!! Ill b th...,negative,_ wockeez be missin u guy by a day ! ! ill b t...,110,wockeez guy day b butttt 18th blockkkkk,,"u june 18th,missin u guy,u guy,june 18th,_ woc..."


In [18]:
submission['selected_text'] =  test['Aspect_Terms'] + ' ' + test['Sentiment_terms']  + ' ' +test['Pytextrank_keyword']

In [19]:
submission.head()

,textID,selected_text
0,11aa4945ff,"wish malta, - wish"
1,fd1db57dc0,"md marathon,done.haha . house md marathon ul..."
2,2524332d66,family concern that family
3,0fb19285b2,"need tho worry tooo tooo many follower tho,no ..."
4,e6c9e5e3ab,"26th, 26th february"


In [20]:
submission.shape

(3535, 2)